In [9]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder



AttributeError: Module 'numpy.core' has no attribute 'numerictypes'

In [ ]:
dataset_path = 'dataset_anomali.csv'
data = pd.read_csv(dataset_path)

print(data)

     CUST_ID      BALANCE  BALANCE_FREQUENCY  PURCHASES  ONEOFF_PURCHASES  \
0     C10001    40.900749           0.818182      95.40              0.00   
1     C10002  3202.467416           0.909091       0.00              0.00   
2     C10003  2495.148862           1.000000     773.17            773.17   
3     C10004  1666.670542           0.636364    1499.00           1499.00   
4     C10005   817.714335           1.000000      16.00             16.00   
...      ...          ...                ...        ...               ...   
8945  C19186    28.493517           1.000000     291.12              0.00   
8946  C19187    19.183215           1.000000     300.00              0.00   
8947  C19188    23.398673           0.833333     144.40              0.00   
8948  C19189    13.457564           0.833333       0.00              0.00   
8949  C19190   372.708075           0.666667    1093.25           1093.25   

      INSTALLMENTS_PURCHASES  CASH_ADVANCE  PURCHASES_FREQUENCY  \
0       

In [ ]:
# Parameters for DE and DBSCAN optimization
MaxIt = 10  # Max number of iterations for DE (lowered for demonstration)
nPop = 10   # Population size
nVar = 2    # Number of decision variables (MinPts and Eps)
bounds = [(2, 20), (0.1, 2)]  # Bounds for MinPts and Eps

# Initialize the population
Population = np.zeros((nPop, nVar))
for i in range(nPop):
    Population[i, :] = [np.random.randint(bounds[0][0], bounds[0][1]),  # MinPts
                        np.random.uniform(bounds[1][0], bounds[1][1])]  # Eps

# Fitness function
def fitness_function(individual, data):
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(data[['BALANCE', 'BALANCE_FREQUENCY', 'PURCHASES', 'ONEOFF_PURCHASES']])
    dbscan = DBSCAN(eps=individual[1], min_samples=int(individual[0])).fit(scaled_data)
    labels = dbscan.labels_
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    if n_clusters > 1:
        silhouette_avg = silhouette_score(scaled_data, labels)
    else:
        silhouette_avg = -1
    return silhouette_avg, {'MinPts': individual[0], 'Eps': individual[1], 'nClusters': n_clusters}

# DE operations
def mutate(individuals, target_idx, F=0.8):
    idxs = [idx for idx in range(len(individuals)) if idx != target_idx]
    a, b, c = np.random.choice(idxs, 3, replace=False)
    mutant = individuals[a] + F * (individuals[b] - individuals[c])
    return np.clip(mutant, [bounds[0][0], bounds[1][0]], [bounds[0][1], bounds[1][1]])

def crossover(target, mutant, CR=0.7):
    cross_points = np.random.rand(nVar) < CR
    trial = np.where(cross_points, mutant, target)
    return trial

def select(individual, trial, data):
    fitness_ind, _ = fitness_function(individual, data)
    fitness_trial, _ = fitness_function(trial, data)
    return trial if fitness_trial > fitness_ind else individual

# DE main loop
best_sol = None
best_fitness = -np.inf

for it in range(MaxIt):
    for i in range(nPop):
        target = Population[i]
        mutant = mutate(Population, i)
        trial = crossover(target, mutant)
        selected = select(target, trial, data)
        Population[i] = selected

        fitness, sol = fitness_function(selected, data)
        if fitness > best_fitness:
            best_fitness = fitness
            best_sol = sol

# Output the best solution found
print("Best Solution:", best_sol)
print("Best Fitness:", best_fitness)

Best Solution: {'MinPts': 3.6, 'Eps': 1.9011047307539504, 'nClusters': 2}
Best Fitness: 0.8693743384528552


In [ ]:
import matplotlib.pyplot as plt

# Use the best parameters from the previous test as an example
MinPts = best_sol[0]
Eps = best_sol[1]

# Run DBSCAN with these parameters on the cleaned dataset
dbscan = DBSCAN(eps=Eps, min_samples=int(MinPts))
y_pred = dbscan.fit_predict(data[['BALANCE', 'BALANCE_FREQUENCY', 'PURCHASES', 'ONEOFF_PURCHASES']])

# Plot the clusters
plt.figure(figsize=(10, 6))
plt.scatter(data['BALANCE'], data['BALANCE_FREQUENCY'], c=y_pred, cmap='viridis', marker='o', s=50, alpha=0.6)
plt.title('DBSCAN Clustering')
plt.xlabel('BALANCE')
plt.ylabel('BALANCE_FREQUENCY')
plt.colorbar(label='Cluster Label')
plt.show()

KeyboardInterrupt: 

In [ ]:
# Filter out anomalies from the dataset
filtered_dataset = data[~anomalies]

# Plotting the clusters after removing anomalies
plt.figure(figsize=(10, 6))
# Since the anomalies have been removed, all remaining data points belong to some cluster
plt.scatter(filtered_dataset['Longitude'], filtered_dataset['Latitude'], c='blue', marker='o', s=50, alpha=0.6, label='Clustered Data')
plt.title('DBSCAN Clustering without Anomalies')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.legend()
plt.show()

In [ ]:
def fitness_anomaly_detection(position, data):
    # Extract MinPts and Eps from the position
    MinPts, Eps = position

    # Run DBSCAN
    dbscan = DBSCAN(eps=Eps, min_samples=int(MinPts))
    labels = dbscan.fit_predict(data[['BALANCE', 'BALANCE_FREQUENCY', 'PURCHASES', 'ONEOFF_PURCHASES']])

    # Identify anomalies
    anomalies = labels == -1
    n_anomalies = np.sum(anomalies)

    # Calculate a simple fitness metric: proportion of anomalies detected
    # Note: This is a placeholder metric. In practice, the fitness function should
    # be tailored to the specific requirements of anomaly detection for the dataset.
    fitness_value = n_anomalies / len(labels)

    return fitness_value, {'MinPts': MinPts, 'Eps': Eps, 'Anomalies': n_anomalies}

# Example: Test the fitness function with initial parameters
test_position = [MinPts, Eps]  # Example parameters (MinPts, Eps)
fitness_value, details = fitness_anomaly_detection(test_position, data)
fitness_value, details

(0.6526256983240224, {'MinPts': 3.6, 'Eps': 10, 'Anomalies': 5841})